## Process WikiCorp Dataset to have a large vocab

In [1]:
import inflect,os,sys,re
from utils.sparse_utils import saveSparseHDF5
from utils.misc import savePickle
p = inflect.engine()

In [2]:
import numpy as np
from utils.sparse_utils import loadSparseHDF5

In [3]:
from load import loadDataset
dset1 = loadDataset('scws')
dset2 = loadDataset('wordsim353')
dset3 = loadDataset('imdb')
dset4 = loadDataset('sst_fine')
dset5 = loadDataset('rotten_tomatoes')
dset6 = loadDataset('sst_binary')
def wParse(k):
    ws = re.sub('[^\w\s]','', k.strip())
    ws = ws.lower()
    return ws
vocab_emb = set([wParse(dset1[k]['w1']) for k in dset1]+[wParse(dset1[k]['w2']) for k in dset1]
               +[wParse(k) for k in dset3['word2idx'].keys()]+[wParse(k) for k in dset4['word2idx'].keys()]
               +[wParse(k) for k in dset5['word2idx'].keys()])
print len(vocab_emb),' words needed'

Loading...
Read  1  objects
Read  1  objects
Read  1  objects
Read  1  objects
171834  words needed


In [ ]:
print dset6['word2idx'].keys()

In [ ]:
#print set(dset6['word2idx'].keys()).difference(set(dset5['word2idx'].keys()))
v_imdb = dset3['word2idx'].keys()
v_sst = dset4['word2idx'].keys()
v_rt = dset5['word2idx'].keys()
def cleanVocab(word):
    ws  = re.sub('[^\w\s-]','', word.strip())
    return ws
newvocab= set()
for w in v_imdb:
    neww = cleanVocab(w)
    if neww in newvocab:
        print w,neww9
    else:
        newvocab.add(neww)
    

In [ ]:
print v_sst

In [4]:
#Load wikicorp vocab
assert os.path.exists('./wikicorp/WestburyLab.wikicorp.201004.feat'),'Feature file not found'
with open('./wikicorp/WestburyLab.wikicorp.201004.feat','r') as f:
    vocab = [k.strip().split(' ')[0] for k in f.readlines()]
print len(vocab)
vocab_arr = np.array(vocab)

2002178


In [5]:
#The number of singular nouns (flags->flag)
vlist  = []
w2idx  = {}

for idx,v in enumerate(vocab):
    sv = p.singular_noun(v)
    if sv:
        vlist.append(sv)
        w2idx[sv] = idx
    else:
        vlist.append(v)
        w2idx[v]  = idx
    
vocab_singular_only = set(vlist)
vocab_singular_list = np.array(vlist)
print vocab_singular_list.shape

(2002178,)


In [7]:
#Find all the words they map to
from_vocab = []
from_vocab_singular = []
absent     = []
mapIdToIdx = {}
for w in vocab_emb:
    if w in vocab:
        idxlst_vocab = np.where(w==vocab_arr)[0].tolist()
        #assert len(idxlst_vocab)==1,'nsd'
        mapIdToIdx[w] = 'v_'+str(idxlst_vocab[0])
        from_vocab  += idxlst_vocab
    elif w in vocab_singular_only:
        idxlst = np.where(w==vocab_singular_list)[0].tolist()
        #assert len(idxlst)==1,'nsd'
        from_vocab_singular+=idxlst
        mapIdToIdx[w] = 's_'+str(idxlst[0])
    else:
        print w,
        absent.append(w)
print ''

 selfexplanatorywhich dahh shaudenfraud ultimatley ctomvelu mooded kalithis loopits zannettis awakecutting colorbeautiful bareaed thoughshould 8mms backmight discussionquestion 71202 bravora yadayadayada hoitytoityness iveneverusedmyvoicebefore howinthehell systemswonderful believabilitythe lyonlisa jewswhen cheezinfested 270 274 275 sh1t actressesits mdmireles1295 noirthe backthats asleepit originalitynot oneher fistfulyojimbo azariawhats producermoody oppositejust oscarhungry captaincyborgmad somedaybut dafoethats nrashootemup tootling anycow kibitzergeneral etherything installmentkilljoy ultralovely rightincluding e7 murphywhen damonjude 1100ad exviet horrornongraphical e3 enchelada supermother trelkovski drymouthed 7ft gigglefits akanightmare lifede goodwe postspielberg suspensethen monoduologues friendsitll moviesnowadays ludicrosity listenit casteffectsboredomexcitementand monstersplatting familiarness curlywannabegonecompletelywrong utterancestrek bsgat paladinwho okcould modes1

IndexError: list index out of range

In [ ]:
print len(set(from_vocab))
print len(set(from_vocab_singular))
print len(set(from_vocab+from_vocab_singular))
idx_to_preserve = from_vocab+from_vocab_singular
print 'Preserving ',len(idx_to_preserve),' words from original vocab'

In [ ]:
data = loadSparseHDF5('dataset','./wikicorp/WestburyLab.wikicorp.201004.h5')
counts = np.array(data.sum(0)).squeeze().astype(int)
print counts.shape

In [ ]:
print 'See how frequent the words are: ',[counts[k] for k in idx_to_preserve]
print counts.shape

In [ ]:
MAXVOCAB   = 20000
sorted_idx = list(set(np.argsort(counts)[-MAXVOCAB:].tolist() + idx_to_preserve))
print np.sort(counts[sorted_idx])

print  len(sorted_idx),np.max(sorted_idx),np.min(sorted_idx),len(vocab)
#Add vectors corresponding to embedding words

In [ ]:
#Check absentees
aidx = []
for idx in idx_to_preserve: 
    if idx not in sorted_idx:
        aidx.append(idx)
print len(aidx),len(idx_to_preserve)

In [ ]:
#Number of words we're double counting ~ 4k should be OK
subset_w = [vocab[i] for i in sorted_idx]
dblct    = []
for w in subset_w:
    if p.singular_noun(w) in subset_w:
        dblct.append(w)
print len(dblct)

In [ ]:
data_subset = data.tocsc()[:,sorted_idx].tocsr()

In [ ]:
features_subset = [vocab[k] for k in sorted_idx]

In [ ]:
features_subset_singular = []
for w in features_subset:
    kk = p.singular_noun(w)
    if kk:
        features_subset_singular.append(kk)
    else:
        features_subset_singular.append(w)

In [ ]:
features_subset          = np.array(features_subset)
features_subset_singular = np.array(features_subset_singular)

In [ ]:
#Check that all the embeddings are availdble here (Except 3)
#Find all the words they map to
mapIdx = {}
for w in vocab_emb:
    if w in features_subset:
        idxlst_vocab = np.where(w==features_subset)[0].tolist()
        assert len(idxlst_vocab)==1,'nsd'
        mapIdx[w] = idxlst_vocab[0]
    elif w in features_subset_singular:
        idxlst = np.where(w==features_subset_singular)[0].tolist()
        assert len(idxlst)==1,'nsd'
        mapIdx[w] = idxlst[0]
    else:
        print w, 'not found'

In [ ]:
#csr_matrix((data, indices, indptr), shape=(3, 3)).toarray()
print data_subset.max(),data_subset.shape
doccts = data_subset.max(1).toarray().squeeze()
docs_keep_idx = np.where(doccts>5)[0]
print docs_keep_idx.shape

data_subset_minlen = data_subset[docs_keep_idx]
print data_subset_minlen.shape

In [ ]:
np.sort(np.array(data_subset_minlen.sum(1)).squeeze())

In [ ]:
#Restrict the documents
np.random.seed(1)
shufidx = np.random.permutation(data_subset_minlen.shape[0])

test_idx    = shufidx[:10000]
valid_idx   = shufidx[10000:12000]
train_idx   = shufidx[12000:]

TRAIN = data_subset_minlen[train_idx]
VALID = data_subset_minlen[valid_idx]
TEST  = data_subset_minlen[test_idx]
print TRAIN.shape, VALID.shape, TEST.shape
print np.sort(np.array(TRAIN.sum(1)).squeeze()).astype(int), np.sort(np.array(VALID.sum(1)).squeeze()), np.sort(np.array(TEST.sum(1)).squeeze())

In [ ]:
cts_train = np.array(TRAIN.sum(0)).squeeze()
for k in mapIdx.values():
    if cts_train[k]<2:
        print features_subset[k],features_subset_singular[k],cts_train[k]

In [ ]:
os.system('rm -rf ./wikicorp/data.h5 ./wikicorp/misc.pkl')
saveSparseHDF5(TRAIN, 'train', './wikicorp/data.h5')
saveSparseHDF5(VALID, 'valid', './wikicorp/data.h5')
saveSparseHDF5(TEST,  'test' , './wikicorp/data.h5')
savePickle([mapIdx,features_subset,features_subset_singular],'./wikicorp/misc.pkl')